# Serialización de objetos

Como revisamos en el material anterior, toda la información que almacena un computador se guarda en base a bits (esto es, ceros y unos). Por ejemplo, imaginemos que buscamos guardar una estructura de datos o una instancia de una clase. De alguna forma, estos datos —que probablemente no tienen una estructura lineal— deben ser guardados como una serie (o secuencia) de bytes. Aquí es cuando aparece el término de **serialización**.

Este concepto se refiere al procedimiento de transformar cualquier objeto en una secuencia o serie de _bytes_. Esto nos permite almacenar el objeto de forma persistente, por ejemplo en un archivo o una base de datos. También nos permite enviar el objeto a otros computadores y programas. 

## `pickle`

El módulo `pickle` de Python nos permite serializar cualquier objeto. Este módulo nos ofrece el método `dumps`, que nos permite serializar un objeto; y el método `loads`, que sirve para deserializar el objeto serializado retornando el objeto original.

In [1]:
import pickle

tupla = ("a", 1, 3, "hola")
serial = pickle.dumps(tupla)
print(serial)
print(type(serial))
print(pickle.loads(serial))
 

b'\x80\x03(X\x01\x00\x00\x00aq\x00K\x01K\x03X\x04\x00\x00\x00holaq\x01tq\x02.'
<class 'bytes'>
('a', 1, 3, 'hola')


`pickle` también nos ofrece los métodos `dump` y `load` (casi el mismo nombre que antes pero sin la _s_). Estos métodos también serializan y deserializan, pero a través de archivos: `dump` guarda un archivo con el objeto serializado y `load` carga el archivo de vuelta.

In [2]:
lista = [1, 2, 3, 7, 8, 3]

with open("mi_lista", 'wb') as file:
    pickle.dump(lista, file)

with open("mi_lista", 'rb') as file:
    mi_lista = pickle.load(file)

# Esto generaría un error si el objeto que cargamos no es igual al que guardamos
assert mi_lista == lista 

`pickle` es un módulo no seguro. Esto implica que **nunca** deben cargar un archivo `pickle` cuando no conocen su procedencia, ya que éste podría ejecutar código malicioso en sus computadores. No entraremos en detalles sobre cómo inyectar código a través del módulo `pickle`, pero los interesados pueden revisar este [enlace](http://www.cs.jhu.edu/~s/musings/pickle.html). 

Cuando `pickle` trata de serializar un objeto, lo primero que hará es verificar que el objeto que se quiere serializar sea de una clase que tenga implementado el método `__getstate__`. Este método debe retornar un diccionario con los atributos que se quieren serializar. Si `__getstate__` no estuviese implementado, entonces `pickle` guardará el atributo `__dict__` del objeto. Recuerden que el atributo `__dict__` es un diccionario que guarda todos los atributos y métodos de un objeto. Por ejemplo, `o.atributo` es equivalente a `o.__dict__["atributo"]` y `o.atributo = 42` es equivalente a `o.__dict__["atributo"] = 42`.

El método `__getstate__` nos permite personalizar la serialización del objeto. Usando este método podemos crear un diccionario que contenga solo la información que deseamos guardar. 

In [3]:
class Persona:
    
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"
        
    def __getstate__(self):
        # Retorna el estado actual del objeto, para que sea serializado por pickle
        # Aquí creamos una copia del diccionario actual, para modificar la copia 
        # y no el objeto original
        nueva = self.__dict__.copy()
        nueva.update({"mensaje" : "¡Me están serializando!"})
        return nueva

m = Persona("Juan", 30)
print(m.mensaje)
serial = pickle.dumps(m)
m2 = pickle.loads(serial)
print(serial)
# el objeto original sigue igual
print(m2.mensaje)
print(m.mensaje)

No pasa nada
b'\x80\x03c__main__\nPersona\nq\x00)\x81q\x01}q\x02(X\x06\x00\x00\x00nombreq\x03X\x04\x00\x00\x00Juanq\x04X\x04\x00\x00\x00edadq\x05K\x1eX\x07\x00\x00\x00mensajeq\x06X\x19\x00\x00\x00\xc2\xa1Me est\xc3\xa1n serializando!q\x07ub.'
¡Me están serializando!
No pasa nada


De forma análoga, podemos personalizar la **deserialización**. Para esto debemos implementar el método `__setstate__`, que se ejecutará cada vez que llamemos a `load` o `loads`. El método `__setstate__` recibe como argumento el diccionario que representa el estado del objeto que fue serializado. Luego debe asignarlo al diccionario del objeto `self.__dict__ = diccionario_con_estado`. Esto no impide que se realicen otras acciones que modifiquen `diccionario_con_estado` antes o después de la asignación.

Si el método `__setstate__` no estuviese implementado, entonces se asignará al `__dict__` del objeto el estado deserializado sin realizar otras acciones adicionales.

In [6]:
class Persona:
    
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
        self.mensaje = "No pasa nada"
        
    def __getstate__(self):
        nueva = self.__dict__.copy()
        nueva.update({"mensaje": "¡Me están serializando!"})
        # esto es lo que será serializado por pickle
        return nueva 

    def __setstate__(self, state):
        print(state, "state")
        print("Objeto recién deserializado, actualizando su estado\n")
        state.update({"nombre": state["nombre"] + " deserializado"})
        print(state)
        self.__dict__ = state
    
m = Persona("Juan", 30)
print(m.nombre)
serial = pickle.dumps(m)
m2 = pickle.loads(serial)
print(m2.nombre)

Juan
{'nombre': 'Juan', 'edad': 30, 'mensaje': '¡Me están serializando!'} state
Objeto recién deserializado, actualizando su estado

{'nombre': 'Juan deserializado', 'edad': 30, 'mensaje': '¡Me están serializando!'}
Juan deserializado


Una aplicación práctica de los métodos `__getstate__` y `__setstate__` es cuando necesitamos serializar un objeto que contiene un atributo que depende de las condiciones actuales del programa. Por ejemplo, imaginemos que un objeto mantiene una conexión a una base de datos. Supongamos, además, que esta conexión tiene atributos relacionados con el estado del programa: tiempo de conexión, puertos utilizados, _host_, entre otros. Cuando guardamos el objeto, deberíamos eliminar la conexión, ya que este atributo no podrá ser utilizado en otra instancia del programa. Para lograr esto usamos el método `__getstate__`. 

Cuando se cargue el mismo objeto desde el archivo serializado, será necesario volver a crear la conexión con las condiciones del programa. Para realizar esto tendremos que implementar `__setstate__`.

# Serialización de objetos con JSON

Una de las desventajas de los objetos serializados con `pickle` es que sólo pueden ser deserializados por otros programas escritos en Python. Por otra parte, **JSON** (JavaScript Object Notation) es un formato estándar de intercambio de datos que puede ser interpretado por muchos lenguajes. JSON además es _human-readable_, es decir, puede ser fácilmente leído y entendido por humanos. El formato en que almacena la información es similar a los diccionarios de Python.

En JSON sólo es posible serializar instancias de `int`, `str`, `float`, `dict`, `bool`, `list`, `tuple` y `NoneType`. Sin embargo, no es posible serializar funciones o instancias de otras clases.
En Python, existe un módulo llamado `json` que provee métodos para serializar objetos en el  formato JSON. Este módulo provee una interfaz similar a la de `pickle` (métodos `dump`(`s`) y `load`(`s`)). 

In [5]:
import json

class Persona:
    
    def __init__(self, nombre, edad, estado_civil):
        self.nombre = nombre
        self.edad = edad
        self.estado_civil = estado_civil
        self.idn = next(Persona.gen)

    def get_id():
        cont = 1
        while True:
            yield cont
            cont += 1

    gen = get_id()
            
p = Persona("Juan", 35, "Soltero")
json_string = json.dumps(p.__dict__)
print("datos en formato JSON: ")
print(json_string)
print(type(json_string))
print("datos en formato Python: ")
print(json.loads(json_string))
print(type(json.loads(json_string)))

datos en formato JSON: 
{"estado_civil": "Soltero", "nombre": "Juan", "edad": 35, "idn": 1}
<class 'str'>
datos en formato Python: 
{'idn': 1, 'edad': 35, 'estado_civil': 'Soltero', 'nombre': 'Juan'}
<class 'dict'>


`json` tiene una tabla de transformación que pueden revisar en [este link](https://docs.python.org/3/library/json.html#encoders-and-decoders). 
Cuando queremos guardar un objeto como JSON podemos personalizar la transformación utilizando un `json.JSONEncoder`, de forma análoga a como lo hicimos `__getstate__`.  

Para esto debemos crear una clase que hereda de la clase `json.JSONEncoder` y sobreescribir el método `default`:

In [6]:
from datetime import datetime


class PersonaEncoder(json.JSONEncoder):
    
       def default(self, obj):
            # Creamos una serialización personalizada para el
            # el tipo de objeto Persona
            
            if isinstance(obj, Persona):
                return {'Persona_id': obj.idn, 
                        'nombre': obj.nombre, 
                        'edad': obj.edad, 
                        'estado_civil': obj.estado_civil, 
                        'fecha_nac' : datetime.now().year - obj.edad}
            
            # Mantenemos la serialización por defecto para 
            # cualquier otro tipo de objeto
            return super().default(obj)


p1 = Persona("Juan", 37, "Soltero")
p2 = Persona("Jorge", 33, "Casado")
p3 = Persona("Pedro", 24, "Soltero")

print("Serialización default:\n")

# con esto serializamos directamente usando el default
json_string = json.dumps(p1.__dict__)
print(json_string)

# Ahora serializamos usando el método personalizado
print("\nSerialización personalizada:\n")
json_string = json.dumps(p1, cls = PersonaEncoder)
print(json_string)

json_string = json.dumps(p2, cls = PersonaEncoder)
print(json_string)

json_string = json.dumps(p3, cls = PersonaEncoder)
print(json_string)


Serialización default:

{"estado_civil": "Soltero", "nombre": "Juan", "edad": 37, "idn": 2}

Serialización personalizada:

{"edad": 37, "fecha_nac": 1980, "Persona_id": 2, "estado_civil": "Soltero", "nombre": "Juan"}
{"edad": 33, "fecha_nac": 1984, "Persona_id": 3, "estado_civil": "Casado", "nombre": "Jorge"}
{"edad": 24, "fecha_nac": 1993, "Persona_id": 4, "estado_civil": "Soltero", "nombre": "Pedro"}


Cuando queremos transformar un JSON a un objeto Python podemos utilizar los `object_hook`, de forma análoga a como lo hicimos con `__setstate__`. El `object_hook` es un parámetro de los métodos `load` y `loads` que recibe una función que recibe un diccionario y retorna el objeto que queremos. Por ejemplo, si queremos cargar datos `json` en una lista de tuplas en vez de un diccionario:

In [7]:
# En los ejemplos guardamos el json en un string para fines ilustrativos
# pero en la vida real recibirán jsons como archivos.
json_string = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado", "puntaje": 90.5}'
datos = json.loads(json_string, object_hook=lambda dict_obj: [(key, value) for key, value in dict_obj.items()])
print(datos)

[('puntaje', 90.5), ('edad', 34), ('estado_civil', 'casado'), ('nombre', 'Jorge')]


Podemos crear cualquier función y aplicarla a los datos que serán convertidos con `json`:

In [8]:
def funcion(dict_obj):
    lista = []
    for k in dict_obj:
        lista.extend([k, str(dict_obj[k])])
    return lista

json_string = '{"nombre": "Jorge", "edad": 34, "estado_civil": "casado", "puntaje": 90.5}'
datos = json.loads(json_string, object_hook=lambda dict_obj: funcion(dict_obj))
print(datos)

['puntaje', '90.5', 'edad', '34', 'estado_civil', 'casado', 'nombre', 'Jorge']
